# App Usage Data SQL Queries

```sql
CREATE TABLE users
    id INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL;

CREATE TABLE sessions
    id INTEGER PRIMARY KEY,
    userId INTEGER NOT NULL FOREIGN KEY REFERENCES users(id),
    duration DECIMAL NOT NULL;

CREATE TABLE transactions
    id INTEGER PRIMARY KEY,
    userId INTEGER NOT NULL FOREIGN KEY REFERENCES users(id)
    productId INTEGER,
    originDate TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    description VARCHAR(30),
    qty DECIMAL;
```

#### User data and their app usage data are kept in the tables above. Each user can have multiple sessions. 
#### Write a SQL query that: 
 - selects the ‘userIds’ with the top 50 average session durations
 - selects their full name from the user table as a single new column

<b> Your query or pseudo query here: </b>

```sql
WITH avg_duration AS (
    SELECT 
        userId,
        AVG(duration) AS average_duration
    FROM 
        sessions
    GROUP BY 
        userId
)

SELECT 
    avg_duration.UserId, 
    CONCAT(users.firstName, ' ', users.lastName) AS 'Full Name'
FROM 
    avg_duration 
INNER JOIN 
    users ON users.id = avg_duration.userId 
ORDER BY 
    avg_duration.average_duration DESC
LIMIT 50 
```


Returns: 

```
userId | Full Name
------------------
5      | Sam Smith
2      | Sara Johnson 
```

### Bonus:  
<b> Add in a new column for each user and flagging their 3rd longest duration with 1, otherwise put 0 (put 0 if they have fewer than 3 sessions) </b>


```sql 
WITH tmp_table AS (
  SELECT 
	sessions.*,
    DENSE_RANK() OVER(PARTITION BY userId ORDER BY duration) AS ranking
FROM 
	sessions
)
   
SELECT 
	id,
    userId,
    duration,
    (CASE 
     	WHEN ranking = 3 THEN 1 
       	ELSE 0 
     END) AS 'Third_longest_dur_flag'
FROM
	tmp_table
```

Returns: 

```
id | userId | Duration | Third_longest_dur_flag
---------------------------------------------
1  |   5    |    25    |     0
2  |   5    |    26    |     0
3  |   5    |    27    |     1
4  |   5    |    28    |     0
```